# 네이버 뉴스 키워드 즐겨찾기.

GLIF에선 뉴욕마켓워치 등 꾸준히 보는 뉴스가 있다. 

이를 매번 네이버 뉴스에서 각각 검색해 보는 것은 번거롭다. 

이러한 뉴스들은 보통 뉴스 제목에 < 뉴욕마켓워치 > 와 같이 특별히 식별 가능한 텍스트가 들어있는 경우가 많다. 

따라서 이러한 텍스트를 필요한 만큼 저장해놓고 글리프 홈페이지에서 한꺼번에 모아볼 수 있도록 article_scraper을 만들었다. 

In [26]:
import pandas as pd
from urllib.parse import quote_plus
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import re

In [82]:
## 관심있는 뉴스의 식별 텍스트들

id_texts = []
while 1:
    id_text = input("뉴스 제목에서 식별 가능한 텍스트를 입력해주세요. 입력이 끝나면 빈칸을 입력해주세요.")
    
    if id_text == "":
    if id_text == []:
        print("식별 텍스트를 하나 이상 입력해주세요.")
        continue
    else:
        break
    
    id_company = input("입력한 키워드 {0}의 언론사도 입력해주세요".format(id_text))
    id_company = id_company.strip()
    
    id_texts.append((id_text, id_company))

네이버에 기사 검색한다고 생각하고 입력해주세요


뉴스 제목에서 식별 가능한 텍스트를 입력해주세요. 입력이 끝나면 빈칸을 입력해주세요. <뉴욕마켓워치>
입력한 키워드 <뉴욕마켓워치>의 언론사도 입력해주세요 연합인포맥스
뉴스 제목에서 식별 가능한 텍스트를 입력해주세요. 입력이 끝나면 빈칸을 입력해주세요. bdi
입력한 키워드 bdi의 언론사도 입력해주세요 아이투자
뉴스 제목에서 식별 가능한 텍스트를 입력해주세요. 입력이 끝나면 빈칸을 입력해주세요. 
입력한 키워드 의 언론사도 입력해주세요 


In [70]:
id_texts_encoded = [quote_plus(x) for x in id_texts]
id_texts_encoded

['%3C%EB%89%B4%EC%9A%95%EB%A7%88%EC%BC%93%EC%9B%8C%EC%B9%98%3E',
 'bdi+%EC%95%84%EC%9D%B4%ED%88%AC%EC%9E%90']

In [71]:
test_text = "<뉴욕마켓워치>"
test_text_encoded = quote_plus(test_text) # encode url. 
test_text_encoded

'%3C%EB%89%B4%EC%9A%95%EB%A7%88%EC%BC%93%EC%9B%8C%EC%B9%98%3E'

In [72]:
naver_period = 4 # 네이버 뉴스 검색 기간
# 1: 1주
# 2: 1개월
# 3: 전체
# 4: 오늘 

In [81]:
url = 'https://search.naver.com/search.naver?where=news&query={0}&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd={1}&ds=&de=&docid=&nso=so%3Ar%2Cp%3A1w%2Ca%3Aall&mynews=0&mson=0&refresh_start=0&related=0'.format(test_text_encoded, naver_period)

html = urlopen(url)
html = bs(html.read(), 'html.parser')

news_list = html.find_all('li', {'id':re.compile('sp_nws')})
#sp_nws1 > dl > dt > a

news_titles = [(x.dl.dt.a.attrs['title'], x.dl.dt.a.attrs['href']) for x in news_list]

filtered_news_lists = list(filter(lambda x: test_text in x[0], news_titles))
filtered_news_lists

[('<뉴욕마켓워치> 英 브렉시트 합의안 부결…주가↑달러↑유가↑',
  'http://news.einfomax.co.kr/news/articleView.html?idxno=4012039')]

# todos:

1. 복수의 검색어에 대해 iterate 하여 total_filtered_news_list 뽑아내기. 
2. url에 대해서 첫 page만 돌지 말고 전체 페이지 돌기. 
3. naver_period 조절하여 오늘만 말고 여러 기간에 대해 검색 가능하게 바꾸기. 
    - 추후 1일, 1주, 1개월 식이 아닌 기간을 customizable 하게 하여 default를 today로 두는 것 고려. 